In [9]:
import pandas as pd
import numpy as np
import os

parquet_data = [f"data/{file}" for file in os.listdir("data") if file.endswith(".parquet")]
df_list = []
for file in parquet_data:
    df = pd.read_parquet(file)
    df["Ticker"] = file.split("/")[1].replace(".parquet", "")
    df_list.append(df)

df = pd.concat(df_list)

In [13]:
df["Error"] = df["Close"] - df["Prediction"]
df["SQE"] = df["Error"]**2
df["LogError"] = np.log(df["Close"]) - np.log(df["Prediction"])
df["PercentageError"] = df["Prediction"]/df["Close"] - 1
df.groupby("Ticker")[["SQE", "LogError", "PercentageError"]].mean()

,SQE,LogError,PercentageError
Ticker,,,
BBAS3.SA,7.434110e-01,-0.003276,0.003792
BTC-USD,8.280692e+06,-0.002304,0.002795
CMIG4.SA,7.939738e-02,0.004713,-0.004341
CSMG3.SA,4.137075e-01,0.002238,-0.001833
ISAE4.SA,4.320994e-01,0.000295,0.000113
ITUB4.SA,7.379193e-01,0.002677,-0.002366
PETR4.SA,9.479703e-01,-0.000201,0.000721
SAPR11.SA,1.017570e+00,0.012651,-0.012110


In [ ]:
from utils import plot_wealth

money = [1000]
quotas = [0]
wealth = [money[0]]

ticker = 'CSMG3.SA'
ticker_df = df.query(f"Ticker == '{ticker}'")
close_sequence = ticker_df["Close"].to_numpy()
prediction_sequence = ticker_df["Prediction"].to_numpy()

last_buy = float("inf")

for i in range(prediction_sequence.shape[0] - 1):
    close = close_sequence[i]
    prediction = prediction_sequence[1 + i]
    pct_change_pred = prediction/close - 1
    last_buy_pct_change = close/last_buy - 1
    # print(prediction, close, pct_change_pred, pct_change_pred > 0.01)

    if pct_change_pred > 0.01:
        quotas.append(quotas[i] + money[i] // close)
        money.append(money[i] % close)
        last_buy = close

    elif pct_change_pred < -0.01:
        money.append(money[i] + quotas[i]*close)
        quotas.append(0)
    
    else:
        money.append(money[i])
        quotas.append(quotas[i])
    
    wealth.append(money[i + 1] + quotas[i + 1]*close)
plot_wealth(ticker_df["Date"].to_numpy(), [wealth], ["Wealth"], f"{ticker}", f"{ticker}_Wealth.png")